# Check that the Notebook Instance is set up correctly

In [ ]:
import boto3

region = boto3.Session().region_name
session = boto3.session.Session()

ec2 = boto3.Session().client(service_name='ec2', region_name=region)
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Check Notebook Instance Name and Instance Type

In [ ]:
import json

with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
    notebook_instance_name = data['ResourceName']

print(notebook_instance_name)

# Check Notebook Instance Settings

In [ ]:
response = sm.describe_notebook_instance(
        NotebookInstanceName=notebook_instance_name
)
print(response)

In [ ]:
try: 
    notebook_instance_arn = response['NotebookInstanceArn']
    print('[OK] Notebook Instance ARN: {}\n'.format(notebook_instance_arn))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[STOP] Could not find the notebook instance ARN in {}.'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
try: 
    subnet_id = response['SubnetId']
    print('[OK] Subnet ID: {}'.format(subnet_id))
except: 
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[STOP]: YOU NEED TO SET UP THE INSTANCE WITH VPC AND SUBNET. Could not find the SubnetId in {}.'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
try: 
    security_group = response['SecurityGroups']
    print('[OK] Security Group: {}'.format(security_group))
except: 
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[STOP]: YOU NEED TO SET UP THE INSTANCE WITH VPC AND SUBNET. Could not find SecurityGroups in {}.'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
try: 
    direct_internet_access = response['DirectInternetAccess']
    if direct_internet_access == 'Disabled':
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('[STOP]: YOU NEED TO SET UP THE INSTANCE WITH INTERNET ACCESS ENABLED')
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    else:
        print('[OK] Direct Internet Access: {}'.format(direct_internet_access))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[STOP]: CHECK YOU SET A VPC AND HAVE DIRECT INTERNET ACCESS ENABLED')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
min_volume_size=250

In [ ]:
volume_size = 0

try:
    volume_size = response['VolumeSizeInGB']
    int(volume_size)
    if volume_size < min_volume_size: 
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('[STOP]: YOU ONLY HAVE {} GB VOLUME SIZE. YOU NEED TO SET UP YOUR INSTANCE WITH 250 GB VOLUME SIZE'.format(volume_size))
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    else:
        print('[OK] Volume Size: {}'.format(volume_size))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[STOP]: COULD NOT FIND THE VOLUME SIZE IN {}'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
if volume_size >= min_volume_size and direct_internet_access == 'Enabled' and security_group and subnet_id and notebook_instance_arn:
    instance_checks_passed=True
    print('[OK] Please continue.')
else:
    instance_checks_passed=False
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[STOP] WE HAVE IDENTIFIED A MISCONFIGURATION.')
    print('[STOP] STOP/DELETE THIS NOTEBOOK INSTANCE AND CREATE A NEW ONE FOLLOWING THE INSTRUCTIONS')
    print('[STOP] volume_size = {}, direct_internet_access = {}, security_group = {}, subnet_id = {}, notebook_instance_arn = {}'.format(volume_size, direct_internet_access, security_group, subnet_id, notebook_instance_arn))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

# Please Make Sure You Don't See ^^ STOP ^^ Message Above

# _If You Do, Please Stop and Delete This Notebook Instance._ 

# _Create A New Notebook Instance With The Correct Settings (See Instructions)._

In [ ]:
%store instance_checks_passed

In [ ]:
%store

# Stop this Notebook to Release Resources

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();